In [1]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

for _ in range(3):
    try:
        import autosklearn.regression as asc
        break
    except:
        pass
else:
    raise ImportError("failed to import from autosklearn")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (4,776 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fron

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [0]:
#conexão com o Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#cria a pasta para manuseio do Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate


In [0]:
def process_data(df_train, df_test, df_answer):

	features = list(df_test)
	features.append('NU_NOTA_MT')

	df_answer['NU_INSCRICAO'] = df_test['NU_INSCRICAO']

	df_train = df_train[features]

	# Dropando tabelas que não serão utilizadas

	df_train.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	df_test.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	# Processado dados categóricos

	df_train['TP_SEXO'].replace('M', 0, inplace=True)
	df_train['TP_SEXO'].replace('F', 1, inplace=True)

	df_test['TP_SEXO'].replace('M', 0, inplace=True)
	df_test['TP_SEXO'].replace('F', 1, inplace=True)

	# Coloquei as notas como zero pq acredito que representam a nota dos alunos que faltaram.

	df_train['NU_NOTA_CN'].fillna(0, inplace=True)
	df_train['NU_NOTA_CH'].fillna(0, inplace=True)
	df_train['NU_NOTA_LC'].fillna(0, inplace=True)
	df_train['NU_NOTA_MT'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_train['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	df_test['NU_NOTA_CN'].fillna(0, inplace=True)
	df_test['NU_NOTA_CH'].fillna(0, inplace=True)
	df_test['NU_NOTA_LC'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_test['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	# Coloquei como 8 pq acredito que represente uma situação neutra

	df_train['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	df_test['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	# One_hot_enconding em variáveis categóricas

	df_train = pd.get_dummies(df_train, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	df_test = pd.get_dummies(df_test, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	# Utilizando Estatística Descritiva

	notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']

	df_train['KURTOSIS_NOTAS'] = df_train[notas].kurtosis(axis=1)
	df_train['MEAN_NOTAS'] = df_train[notas].mean(axis=1)
	df_train['MEDIAN_NOTAS'] = df_train[notas].median(axis=1)
	df_train['MAD_NOTAS'] = df_train[notas].mad(axis=1)
	df_train['QUANTILE_NOTAS'] = df_train[notas].quantile(axis=1)
	df_train['SEM_NOTAS'] = df_train[notas].sem(axis=1)
	df_train['SKEW_NOTAS'] = df_train[notas].skew(axis=1)
	df_train['STD_NOTAS'] = df_train[notas].std(axis=1)
	df_train['VAR_NOTAS'] = df_train[notas].var(axis=1)
	df_train['AMP_NOTAS'] = df_train[notas].max(axis=1) - df_train[notas].min(axis=1)

	df_train['MEAN_NOTA_CH_LC'] = df_train[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CH_CN'] = df_train[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_train['MEAN_NOTA_CH_REDACAO'] = df_train[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_CN_LC'] = df_train[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CN_REDACAO'] = df_train[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_LC_REDACAO'] = df_train[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)

	notas_red = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

	df_train['KURTOSIS_NOTAS_COMP'] = df_train[notas_red].kurtosis(axis=1)
	df_train['MEAN_NOTAS_COMP'] = df_train[notas_red].mean(axis=1)
	df_train['MEDIAN_NOTAS_COMP'] = df_train[notas_red].median(axis=1)
	df_train['MAD_NOTAS_COMP'] = df_train[notas_red].mad(axis=1)
	df_train['QUANTILE_NOTAS_COMP'] = df_train[notas_red].quantile(axis=1)
	df_train['SEM_NOTAS_COMP'] = df_train[notas_red].sem(axis=1)
	df_train['STD_NOTAS_COMP'] = df_train[notas_red].std(axis=1)
	df_train['VAR_NOTAS_COMP'] = df_train[notas_red].var(axis=1)
	df_train['AMP_NOTAS_COMP'] = df_train[notas_red].max(axis=1) - df_train[notas_red].min(axis=1)

	df_train['MEAN_NOTA_COMP1_COMP2'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP3'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP4'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP5'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP3'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP4'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP5'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP4'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP5'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP4_COMP5'] = df_train[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)  
  
	df_test['KURTOSIS_NOTAS'] = df_test[notas].kurtosis(axis=1)
	df_test['MEAN_NOTAS'] = df_test[notas].mean(axis=1)
	df_test['MEDIAN_NOTAS'] = df_test[notas].median(axis=1)
	df_test['MAD_NOTAS'] = df_test[notas].mad(axis=1)
	df_test['QUANTILE_NOTAS'] = df_test[notas].quantile(axis=1)
	df_test['SEM_NOTAS'] = df_test[notas].sem(axis=1)
	df_test['SKEW_NOTAS'] = df_test[notas].skew(axis=1)
	df_test['STD_NOTAS'] = df_test[notas].std(axis=1)
	df_test['VAR_NOTAS'] = df_test[notas].var(axis=1)
	df_test['AMP_NOTAS'] = df_test[notas].max(axis=1) - df_test[notas].min(axis=1)
  
	df_test['MEAN_NOTA_CH_LC'] = df_test[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CH_CN'] = df_test[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_test['MEAN_NOTA_CH_REDACAO'] = df_test[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_CN_LC'] = df_test[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CN_REDACAO'] = df_test[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_LC_REDACAO'] = df_test[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)

	df_test['KURTOSIS_NOTAS_COMP'] = df_test[notas_red].kurtosis(axis=1)
	df_test['MEAN_NOTAS_COMP'] = df_test[notas_red].mean(axis=1)
	df_test['MEDIAN_NOTAS_COMP'] = df_test[notas_red].median(axis=1)
	df_test['MAD_NOTAS_COMP'] = df_test[notas_red].mad(axis=1)
	df_test['QUANTILE_NOTAS_COMP'] = df_test[notas_red].quantile(axis=1)
	df_test['SEM_NOTAS_COMP'] = df_test[notas_red].sem(axis=1)
	df_test['STD_NOTAS_COMP'] = df_test[notas_red].std(axis=1)
	df_test['VAR_NOTAS_COMP'] = df_test[notas_red].var(axis=1)
	df_test['AMP_NOTAS_COMP'] = df_test[notas_red].max(axis=1) - df_test[notas_red].min(axis=1)

	df_test['MEAN_NOTA_COMP1_COMP2'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP3'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP4'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP5'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP3'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP4'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP5'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP4'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP5'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP4_COMP5'] = df_test[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)

	return df_train, df_test, df_answer

In [0]:
def convert_to_zero(filename):

	data = pd.read_csv(filename+'.csv')

	for i in range(0, data.shape[0]):
		if data['NU_NOTA_MT'][i] < 80:
			data['NU_NOTA_MT'][i] = 0.00

	data.to_csv('/content/drive/ML/data-codenation-challenge/sex_answer.csv', index=False, header=True)

	return data

In [0]:
def main(time, df_train, label, df_test, df_answer):

	classifier = asc.AutoSklearnRegressor(
	    time_left_for_this_task=time+10,
	    per_run_time_limit=time,
	    initial_configurations_via_metalearning=0,
	    resampling_strategy='cv',
	    resampling_strategy_arguments={'folds': 5},
	)

	classifier.fit(df_train.copy(), label.copy())

	classifier.refit(df_train.copy(), label.copy())

	pickle.dump(classifier, open('/content/drive/ML/data-codenation-challenge/sex_automl_regressor.pkl', 'wb'))

	predictions = classifier.predict(df_test)

	df_answer['NU_NOTA_MT'] = np.around(predictions, 2)

	df_answer.to_csv('/content/drive/ML/data-codenation-challenge/sex_automl_answer.csv', index=False, header=True)

In [0]:
import sklearn as sk
import pandas as pd
import numpy as np
import pickle


df_train = pd.read_csv('/content/drive/ML/data-codenation-challenge/train.csv')
df_test = pd.read_csv('/content/drive/ML/data-codenation-challenge/test.csv')
df_answer = pd.DataFrame()

df_train, df_test, df_answer = process_data(df_train, df_test, df_answer)

label = df_train['NU_NOTA_MT']

df_train.drop(['NU_NOTA_MT'], axis=1, inplace=True)

main(60*60*10, df_train, label, df_test, df_answer)

data = convert_to_zero('/content/drive/ML/data-codenation-challenge/sex_automl_answer')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


[WARNING] [2019-04-19 15:33:25,986:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:25,997:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:28,000:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:30,011:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:32,022:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:34,034:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:36,046:EnsembleBuilder(1):c3e7341e906e8331bbfae00b31dfae96] No models better than random - using Dummy Score!
[WARNING] [2019-04-19 15:33:38,058

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
